In [12]:
import pandas as pd
from pyairtable import Api
from sqlalchemy import create_engine

engine = create_engine("postgresql://playground@localhost:5432/postgres")

In [15]:
rfm = pd.read_json('https://extrem.ishukshin.ru/json')
rfm = rfm.convert_dtypes()
rfm.is_ukrainian = rfm.is_ukrainian.astype('boolean')
rfm.head()

,id,name_full,names,is_terr,birth,place,is_ukrainian
0,1,АБАДИЕВ МАГОМЕД МИКАИЛОВИЧ*,[АБАДИЕВ МАГОМЕД МИКАИЛОВИЧ],True,09.11.1982 г.р.,Г. НАЗРАНЬ ЧИАССР,False
1,3,АБАЕВ ИДРИС МОВСАРОВИЧ*,[АБАЕВ ИДРИС МОВСАРОВИЧ],True,02.01.1986 г.р.,С. СУВОРОВ-ЮРТ ГУДЕРМЕССКОГО РАЙОНА ЧИАССР,False
2,6,АБАЗОВ АХМЕД ЭДУАРДОВИЧ*,[АБАЗОВ АХМЕД ЭДУАРДОВИЧ],True,30.08.1992 г.р.,С. КЫЗБУРУН-3 БАКСАНСКОГО РАЙОНА КАБАРДИНО-БАЛ...,False
3,7,АБАЗОВ БЕСЛАН СУПЬЯНОВИЧ*,[АБАЗОВ БЕСЛАН СУПЬЯНОВИЧ],True,15.08.1993 г.р.,С. ШАЛИ ШАЛИНСКОГО РАЙОНА ЧЕЧЕНСКОЙ РЕСПУБЛИКИ,False
4,8,АБАЗОВ МУРАТ АРСЕНОВИЧ*,[АБАЗОВ МУРАТ АРСЕНОВИЧ],True,10.05.1989 г.р.,Г. НАЛЬЧИК КАБАРДИНО-БАЛКАРСКОЙ РЕСПУБЛИКИ,False


In [40]:
api = Api('patoCRTkfxdzWWT0Y.f20c7fccca48bbbbffe5d1d993f8415ebc5fd3b2d95b4a15ab38e590dc949202')
pzk = pd.json_normalize(api.table('app1sDXWAmADet7vo', 'tbl321Qu3l3aQfvYp').all())
pzk.rename(columns={
    'id': 'id',
    'createdTime': 'created_time',
    'fields.++ФИО': 'name',
    'fields.Дело': 'case',
    'fields.Регион' : 'region',
    'fields.++Город' : 'town',
    'fields.Возраст' : 'age',
    'fields.Росфинмониторинг' : 'rfm',
    'fields.Дата включения в список РФМ' : 'date_rfm_added',
    'fields.Дата исключения из списка РФМ' : 'date_rfm_removed'
}, inplace=True)
pzk = pzk.convert_dtypes()
pzk.date_rfm_added = pd.to_datetime(pzk.date_rfm_added)
pzk.date_rfm_removed = pd.to_datetime(pzk.date_rfm_removed)
pzk.created_time = pd.to_datetime(pzk.created_time)
pzk.set_index('id', inplace=True)
pzk.head()

,created_time,case,name,town,age,region,date_rfm_added,rfm,date_rfm_removed
id,,,,,,,,,
rec00XVQtCOoOUVXe,2024-03-29 21:20:24+00:00,Дело о Маяковских чтениях,Арифджанов Назар Рустемович,Москва,25,Москва,NaT,<NA>,NaT
rec02rGNoN09kEzaL,2024-03-29 21:20:24+00:00,Дело против беженца Ильшата Шарафуллина,Шарафуллин Ильшат Маратович,Набережные Челны,48,Республика Татарстан,2023-01-31,True,NaT
rec03WXmsrOycbY9C,2024-03-29 21:20:24+00:00,Дворцовое дело,Коваленко Ренат Газизович,Новосибирск,30,Новосибирская область,2021-03-19,True,2023-10-12
rec03pA4ohGmZXJ9T,2024-03-29 21:20:26+00:00,Дело о подготовке теракта на День победы во вр...,Запорожец Павел Павлович,Скадовск,33,Херсонская область,2022-08-19,True,NaT
rec04cHEq7gWefvMQ,2024-03-29 21:20:26+00:00,Обыски у активистов Объединения перевозчиков Р...,Понеж Сергей,Санкт-Петербург,<NA>,Санкт-Петербург,NaT,<NA>,NaT


In [46]:
with engine.begin() as connection:
    print(pzk.to_sql('pzk', connection, if_exists='replace'))

542


In [53]:
df = pd.read_sql('pzk', engine, index_col='id')
df.head()

,created_time,case,name,town,age,region,date_rfm_added,rfm,date_rfm_removed
id,,,,,,,,,
rec00XVQtCOoOUVXe,2024-03-29 21:20:24+00:00,Дело о Маяковских чтениях,Арифджанов Назар Рустемович,Москва,25.0,Москва,NaT,None,NaT
rec02rGNoN09kEzaL,2024-03-29 21:20:24+00:00,Дело против беженца Ильшата Шарафуллина,Шарафуллин Ильшат Маратович,Набережные Челны,48.0,Республика Татарстан,2023-01-31,True,NaT
rec03WXmsrOycbY9C,2024-03-29 21:20:24+00:00,Дворцовое дело,Коваленко Ренат Газизович,Новосибирск,30.0,Новосибирская область,2021-03-19,True,2023-10-12
rec03pA4ohGmZXJ9T,2024-03-29 21:20:26+00:00,Дело о подготовке теракта на День победы во вр...,Запорожец Павел Павлович,Скадовск,33.0,Херсонская область,2022-08-19,True,NaT
rec04cHEq7gWefvMQ,2024-03-29 21:20:26+00:00,Обыски у активистов Объединения перевозчиков Р...,Понеж Сергей,Санкт-Петербург,NaN,Санкт-Петербург,NaT,None,NaT


In [59]:
df.index

Index(['rec00XVQtCOoOUVXe', 'rec02rGNoN09kEzaL', 'rec03WXmsrOycbY9C',
       'rec03pA4ohGmZXJ9T', 'rec04cHEq7gWefvMQ', 'rec055K5hNqR07YsX',
       'rec05Pq0b6B7Ten5C', 'rec05UDkreFHnmaSr', 'rec05tlZYwGhN6gQk',
       'rec06fRndUamEHCzx',
       ...
       'reczsyFKlko6GbRYU', 'recztuKPMmss3id1n', 'reczuUzCUS1Q0P52C',
       'reczuc4QyL8Lj31NW', 'reczwO2hEvI26qZR4', 'reczwdviDjkOxr254',
       'reczx3M3YwQzj0md6', 'reczxkB0fKK7n6oJo', 'reczycygMgqn3bt9p',
       'reczzNd1D5AM5DsCj'],
      dtype='object', name='id', length=5542)

In [66]:
df.index = df.index.astype('string')

In [70]:
df.sort_index().index.equals(pzk.sort_index().index)

True

In [72]:
df.sort_index().compare(pzk.sort_index())

id
